In [ ]:
##### >START FROM HEREEEE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# add new work arragement column to raw data files
import os
import pandas as pd

# Directory containing all data files and subdirectories
root_directory = '/content/drive/MyDrive/Intro_DS/Datasets/RawData/'

# File name for file_index CSV
file_index_path = '/content/drive/MyDrive/Intro_DS/Datasets/work_argmnt.csv'

# Read the file_index CSV
file_index_df = pd.read_csv(file_index_path)

# Traverse through all files in the directory and subdirectories
for subdir, dirs, files in os.walk(root_directory):
    for file_name in files:
        if file_name.endswith('.csv'):
            # Extract the first integer before the first underscore '_'
            file_prefix = file_name.split('_')[0]

            try:
                file_integer = int(file_prefix)
            except ValueError:
                # If no integer is found, skip the file
                continue

            # Find the matching row in the file_index file based on file_integer
            matching_row = file_index_df[file_index_df['Number'] == file_integer]

            if not matching_row.empty:
                # Get the corresponding work_arrangement value
                work_arrangement_value = matching_row['work_arrangement'].values[0]

                # Read the data file
                data_file_path = os.path.join(subdir, file_name)
                data_df = pd.read_csv(data_file_path)

                # Add the new column 'work_arrangement' and fill it with the same value
                data_df['work_arrangement'] = work_arrangement_value

                # Save the updated DataFrame back to the file (or a new file if needed)
                data_df.to_csv(data_file_path, index=False)
                print(f"Updated {file_name} in {subdir} with work arrangement: {work_arrangement_value}")
            else:
                print(f"No matching file_index for {file_name} in {subdir}")


Updated 1_dataset_linkedin-jobs-scraper_2024-09-12_10-16-49-660.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: On-site
Updated 34_dataset_linkedin-jobs-scraper_2024-09-12_11-07-48-559.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: Remote
Updated 3_dataset_linkedin-jobs-scraper_2024-09-12_10-21-52-488.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: On-site
Updated 63_dataset_linkedin-jobs-scraper_2024-09-12_11-42-01-980.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: Hybrid
Updated 2_dataset_linkedin-jobs-scraper_2024-09-12_10-20-19-485.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: On-site
Updated 22_dataset_linkedin-jobs-scraper_2024-09-12_10-51-05-704.csv in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist with work arrangement: On-site
Upd

In [ ]:
# Main Class

def main():
  #STEP 1 :
  #READ RAW FILES ####
    # Define the paths to the folders containing the CSV files
    folder_paths = [
    '/content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Analyst',
    '/content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Scientist'  # Replace with the actual path of the second folder
]

    # Create an instance of the DataPreprocessing class
   # data_preprocessor = DataPreprocessing(folder_paths)

    # Call the read_files method to combine CSV files
    combined_df = read_files(folder_paths)

    # Save the combined DataFrame to a new CSV file if it's not empty
    if not combined_df.empty:
        combined_df.to_csv('/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_data_file.csv', index=False)
        print("All CSV files have been successfully combined!")
    else:
        print("No data to save.")

    #STEP 2: REMOVE IDENTICAL RAWS
    input_path = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_data_file.csv'
    output_path1 = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_cleaned_1.csv'
    cleaned_df= remove_duplicates(input_path, output_path1)
    print("REMOVED IDENTICAL RAWS!!!")
    # STEP 3: LANGUAGE TRASLATION IF NEEDED
    # Specify the CSV file name here
    # Call the language detection function
    updated_csv_file = add_language_column_and_filter(output_path1)
    without_finnish_csv = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_cleaned_1_without_finnish.csv'
    translated_finnish_csv = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_cleaned_1_finnish_translated.csv'
    combined_csv_file = combine_csv_files(without_finnish_csv, translated_finnish_csv)
    print("TRANSLATED TO ENGLISH!!!")

    # STEP 4: FEATURE ENGINEERING

    cleaned_df='/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_csv_file.csv'
    cleaned_df = pd.read_csv(cleaned_df)
    #output_file_path2 = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_with_data_science_skills.csv'
    keywords_file_path = '/content/drive/MyDrive/Intro_DS/Datasets/keywords.txt'
    # Load the keywords from the text file
    keywords = read_keywords_from_file(keywords_file_path)
    cleaned_df[['required skills', 'Data Base Applications', 'Data Engineering', 'Cloud Computing', 'Soft Skills']] = cleaned_df['description'].apply(lambda desc: pd.Series(extract_info(desc, keywords)))

    print("processed _job_descriptions!!!")
    print(cleaned_df)
    df = cleaned_df

    # STEP 5: JPB APPLICANTS COLUMN PROCESSING
    # Remove the word 'applicants' from the strings in the column
    df['applicationsCount'] = df['applicationsCount'].str.replace(' applicants', '', regex=False)
    # Apply the function to the column
    df['applicationsCount'] = df['applicationsCount'].apply(replace_qualitative_value)
    print(df['applicationsCount'])
    df= categorize_into_intervals(df,'applicationsCount' )
    print(df['applicationsCount_interval'])
    print("Processed applicationsCount and created new column applicationsCount_interval")

    # classify the company level using job description column

    df['company_level'] = df['description'].apply(lambda x: classify_company_level(x, keywords))
    print(df['company_level'])
    print("Processed description and created new column company_level")

    #Location column splite
    # Correct usage of str.split() to split the 'location' column into two new columns
    df[['location_1', 'location_2']] = df['location'].str.split(',', n=1, expand=True)

    # Strip any leading or trailing whitespace
    df['location_1'] = df['location_1'].str.strip()
    df['location_2'] = df['location_2'].str.strip()

    #### processing postedTime column
    # Apply the function to get the number of weeks
    df['displayed_weeks'] = df['postedTime'].apply(convert_to_weeks)
    # Apply the categorization function to get the job_displayedTime
    df['job_displayedTime'] = df['displayed_weeks'].apply(categorize_weeks)
    print("Processed postedTime and created new column job_displayedTime")

    # Convert 'publishedAt' to datetime format
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    # Extract the month as a number from the 'publishedAt' column
    df['published_month'] = df['publishedAt'].dt.strftime('%m')  # Month as number
    print("Processed publishedAt and created new column published_month")

    ### derive type of work with job title
    # Apply the function to the DataFrame
    df[['general_job_category', 'job_title_desc']] = df['title'].apply(lambda x: pd.Series(extract_tokens_and_text(x)))

    # Split the text in 'job_title_desc' into a list of words
    df['job_title_desc_list'] = df['job_title_desc'].str.split()
    print(df['job_title_desc_list'])
    print("Processed title and created new column job_title_desc_list")


    ### process worktype column
    # Apply the function to split and filter text
    df['worktype_desc'] = df['workType'].apply(split_and_filter)
    print(df['worktype_desc'])
    print("Processed workType and created new column worktype_desc")

    ################################ DATA ENGINEERING END  ############################

    ################################## DATA PROFILING PART ############################
    result = data_profiling(df)
    print(result)

    # Add more to data profiling functions if need to data cleaning
    ################################## END OF DATA PROFILING! ########################
    main_save_path = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/'
    cleaned_df = data_profiling(df, duplicates_path='duplicates.csv', summary_path= main_save_path+'data_prifile_before_cleaning.txt')
    cleaned_df.to_csv(main_save_path+'cleaned_dataset_v1.csv', index=False)  # Save the cleaned DataFrame FULL

    # Create DataFrame without the description column
    df_no_description = cleaned_df.drop(columns=['description'])

    # Save the DataFrame without the description column
    df_no_description.to_csv(main_save_path+'df_no_description.csv', index=False)

    # Create DataFrame with specified columns
    desc_df = df[['applyUrl', 'companyId', 'companyName', 'description']]

    # Save the DataFrame with specified columns
    desc_df.to_csv(main_save_path+'desc_df.csv', index=False)

    # Columns to check for null counts
    columns_to_check = ['required skills', 'Data Base Applications', 'Data Engineering', 'Cloud Computing', 'Soft Skills']

    # Calculate and print null counts for each column
    null_counts = cleaned_df[columns_to_check].isnull().sum()

    # Display null counts
    print(null_counts)

    ################################## Unknown value count ########################
    file_path = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/cleaned_dataset_v1.csv'
    columns_to_check = ['required skills', 'Data Base Applications', 'Data Engineering', 'Cloud Computing', 'Soft Skills']
    unknown_counts = count_unknown_values_in_csv(file_path, columns_to_check)
    print(unknown_counts)



In [ ]:
main()

CSV files found in /content/drive/MyDrive/Intro_DS/Datasets/RawData/Data_Analyst: ['94_dataset_linkedin-jobs-scraper_2024-09-12_12-24-16-811.csv', '121_dataset_linkedin-jobs-scraper_2024-09-12_12-55-08-392.csv', '104_dataset_linkedin-jobs-scraper_2024-09-12_12-35-52-515.csv', '122_dataset_linkedin-jobs-scraper_2024-09-12_12-58-55-894.csv', '93_dataset_linkedin-jobs-scraper_2024-09-12_12-22-16-612.csv', '124_dataset_linkedin-jobs-scraper_2024-09-12_13-02-06-192.csv', '112_dataset_linkedin-jobs-scraper_2024-09-12_12-44-13-244.csv', '92_dataset_linkedin-jobs-scraper_2024-09-12_12-20-14-710.csv', '132_dataset_linkedin-jobs-scraper_2024-09-12_13-12-28-397.csv', '155_dataset_linkedin-jobs-scraper_2024-09-12_13-36-21-416.csv', '154_dataset_linkedin-jobs-scraper_2024-09-12_13-34-23-848.csv', '153_dataset_linkedin-jobs-scraper_2024-09-12_13-32-21-430.csv', '152_dataset_linkedin-jobs-scraper_2024-09-12_13-29-42-478.csv', '174_dataset_linkedin-jobs-scraper_2024-09-12_18-21-04-267.csv', '162_datas

In [ ]:
 ### READ RAW FILES #########################
import pandas as pd
import os

def read_files(folder_paths):
    """
    Reads all CSV files from the specified folder paths and combines them into a single DataFrame.

    Args:
        folder_paths (list): A list of folder paths containing CSV files.

    Returns:
        DataFrame: A combined DataFrame containing all data from the CSV files.
    """
    dataframes = []  # List to store individual DataFrames

    # Loop through each folder
    for folder_path in folder_paths:
        # Get a list of all CSV files in the folder
        csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
        print(f"CSV files found in {folder_path}: {csv_files}")  # Log found CSV files

        # Loop through the list of CSV files and append each DataFrame to the list
        for file in csv_files:
            df = pd.read_csv(os.path.join(folder_path, file))
            if df.empty:
                print(f"Warning: {file} is empty.")
            else:
                dataframes.append(df)
                print(f"Loaded DataFrame from {file}: {df.shape[0]} rows")  # Print the number of rows loaded

    # Check if there are DataFrames to concatenate
    if not dataframes:
        print("No DataFrames to concatenate. Please check your CSV files.")
        return pd.DataFrame()  # Return an empty DataFrame if none found

    # Concatenate all DataFrames into one
    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df


In [ ]:
############# REMOVE DUPLICATES ##############################

import pandas as pd

def remove_duplicates(input_file, output_file):
    """
    Removes duplicate rows from a CSV file and saves the cleaned data.

    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the cleaned CSV file.
    """
    # Load the combined CSV file
    df = pd.read_csv(input_file)

    # Count the total number of records before removing duplicates
    initial_count = len(df)

    # Strip leading/trailing spaces in all string columns
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # Remove identical (duplicate) rows
    df_cleaned = df.drop_duplicates()

    # Count the total number of records after removing duplicates
    final_count = len(df_cleaned)

    # Save the cleaned DataFrame to a new CSV file
    df_cleaned.to_csv(output_file, index=False)

    # Print the counts
    print(f"Number of records before removing duplicates: {initial_count}")
    print(f"Number of records after removing duplicates: {final_count}")
    print(f"Number of duplicate records removed: {initial_count - final_count}")
    return df_cleaned

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=3185910b7ef811d31f53722bf3830c56ab7fe3bffcd80a7f932037e035c84d1e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
pip install googletrans

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=d6bb0cd5aeedf5ed775a7f74b383f369a78fd4580cf6d23422cdc387f7fb1eee
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
   

In [ ]:
 ## STEP 3 : LANGUAGE TRANSLATION ###############
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def add_language_column_and_filter(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Function to detect language of the description
    def detect_language(text):
        try:
            return detect(text)
        except LangDetectException:
            return "unknown"

    # Apply the function to the 'description' column and create a new 'language posted' column
    df['language_posted'] = df['description'].apply(detect_language)

    # Extract records posted in Finnish (language code: 'fi')
    finnish_records = df[df['language_posted'] == 'fi']

    # Save Finnish records to a new CSV file
    finnish_csv_file = csv_file.replace('.csv', '_finnish.csv')
    finnish_records.to_csv(finnish_csv_file, index=False)

    # Drop Finnish records from the original DataFrame
    df_without_finnish = df[df['language_posted'] != 'fi']

    # Save the updated DataFrame without Finnish records to a new CSV file
    updated_csv_file = csv_file.replace('.csv', '_without_finnish.csv')
    df_without_finnish.to_csv(updated_csv_file, index=False)

    # Print the updated CSV headings
    print("Updated CSV Headings (without Finnish records):")
    print(df_without_finnish.columns.tolist())

    # Print the number of records in both CSV files
    print(f"Number of records in the Finnish records CSV file: {len(finnish_records)}")
    print(f"Number of records in the updated CSV file (without Finnish records): {len(df_without_finnish)}")

    return finnish_csv_file, updated_csv_file


import pandas as pd

def combine_csv_files(without_finnish_csv, translated_finnish_csv):
    # Read the CSV files
    df_without_finnish = pd.read_csv(without_finnish_csv)
    df_translated_finnish = pd.read_csv(translated_finnish_csv)

    # Combine the DataFrames
    combined_df = pd.concat([df_without_finnish, df_translated_finnish], ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    combined_csv_file_path = '/content/drive/MyDrive/Intro_DS/MiniProject/outputs/combined_csv_file.csv'
    combined_df.to_csv(combined_csv_file_path, index=False)

    # Print the number of records in the combined DataFrame
    print(f"Number of records in the Combined CSV file: {len(combined_df)}")

    print(f"Combined CSV saved as {combined_csv_file_path}")
    return combined_csv_file_path



In [ ]:
### STEP 4 : FEATURE ENGINEERING PART

# DESCRIPTION COLUMN
import pandas as pd

def read_keywords(file_path):
    """
    Reads keyword categories from a txt file and returns a dictionary of lists.
    """
    keywords = {}
    current_category = None

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('[') and line.endswith(']'):
                current_category = line[1:-1]
                keywords[current_category] = []
            elif current_category:
                keywords[current_category] = [x.strip() for x in line.split(',')]

    return keywords

def extract_info(description, keywords):
    """
    Extracts skills, databases, data engineering, cloud computing, and soft skills
    from the job description based on provided keywords.
    """
    skills, databases, data_engineering, cloud_computing, soft_skills = set(), set(), set(), set(), set()

    # Check for keywords in each category
    for skill in keywords.get('skills_keywords', []):
        if skill.lower() in description.lower():
            skills.add(skill)

    for db in keywords.get('database_keywords', []):
        if db.lower() in description.lower():
            databases.add(db)

    for eng in keywords.get('data_engineering_words', []):
        if eng.lower() in description.lower():
            data_engineering.add(eng)

    for cloud in keywords.get('cloud_computing_words', []):
        if cloud.lower() in description.lower():
            cloud_computing.add(cloud)

    for soft in keywords.get('soft_skills_word', []):
        if soft.lower() in description.lower():
            soft_skills.add(soft)

    # Return a comma-separated list of found keywords
    return (
        ', '.join(skills) if skills else 'Unknown',
        ', '.join(databases) if databases else 'Unknown',
        ', '.join(data_engineering) if data_engineering else 'Unknown',
        ', '.join(cloud_computing) if cloud_computing else 'Unknown',
        ', '.join(soft_skills) if soft_skills else 'Unknown'
    )


In [ ]:
# Number of appliants column processing
# Function to extract the number from the string and subtract 1, or keep the number as is

def replace_qualitative_value(value):
    if isinstance(value, str) and 'Be among the first' in value:
        # Extract the number using string split
        num = int(value.split()[4])  # "Be among the first X applicants" --> extract X
        return num - 1  # Subtract 1 from the number
    elif value.isdigit():  # Check if the value is already a number (as a string)
        return int(value)  # Convert it to integer
    else:
        return value  # Return the value as is if it's not a string or numeric

In [ ]:
import pandas as pd

# Function to categorize job applicants counts into 25-interval ranges up to 200
def categorize_into_intervals(df, column_name):
    # Apply the categorization to the specified column
    df[column_name + '_interval'] = df[column_name].apply(lambda x: categorize_value(x))
    return df  # Return the updated DataFrame

# Categorization logic for each value in the specified column
def categorize_value(value):
    try:
        num = int(value)
        if num <= 25:
            return '0-25'
        elif num <= 50:
            return '26-50'
        elif num <= 75:
            return '51-75'
        elif num <= 100:
            return '76-100'
        elif num <= 125:
            return '101-125'
        elif num <= 150:
            return '126-150'
        elif num <= 175:
            return '151-175'
        elif num <= 200:
            return '176-200'
        else:
            return 'Over 200'  # For values above 200
    except ValueError:
        return value  # In case of non-numeric or unexpected value


In [ ]:
# Function to classify company level based on job description
# Function to read keywords from a file
def read_keywords_from_file(file_path):
    keywords = {}
    with open(file_path, 'r') as file:
        current_category = None
        for line in file:
            line = line.strip()
            if line.startswith("["):  # Check if the line is a category
                current_category = line[1:-1]  # Remove brackets
                keywords[current_category] = []
            elif current_category is not None and line:
                # Add keywords to the current category
                keywords[current_category].extend(line.split(", "))
    return keywords

# Updated classify function
def classify_company_level(description, keywords):
    description = description.lower()

    # Check for startup keywords
    if any(keyword in description for keyword in keywords['company_keywords_startup']):
        return 'Startup'

    # Check for medium-sized company keywords
    elif any(keyword in description for keyword in keywords['company_keywords_medium']):
        return 'Medium-Sized'

    # Check for large-scale company keywords
    elif any(keyword in description for keyword in keywords['company_keywords_large']):
        return 'Large-Scale'

    # If no match, return unknown
    return 'Unknown'


In [ ]:
# derived variables : 'job_displayedTime'  in weeks
import re

# Remove the word 'ago' and convert to a standard format
def convert_to_weeks(time_str):
    if pd.isnull(time_str):
        return None

    # Remove 'ago' and handle the time string
    time_str = time_str.replace(' ago', '')

    # Extract numbers and time units
    match = re.match(r'(\d+)\s*(day|week|month|hour|minute)s?', time_str)
    if match:
        value, unit = match.groups()
        value = int(value)

        # Convert to weeks
        if unit in ['day', 'days']:
            return value / 7
        elif unit in ['week', 'weeks']:
            return value
        elif unit in ['month', 'months']:
            return value * 4  # Approximate a month as 4 weeks
        elif unit in ['hour', 'hours']:
            return value / 168  # 168 hours in a week
        elif unit in ['minute', 'minutes']:
            return value / 10080  # 10080 minutes in a week

    return None


# Define a function to categorize the weeks into the required format
def categorize_weeks(weeks):
    if weeks is None:
        return None
    elif weeks < 1:
        return '1w'
    elif 1 <= weeks < 2:
        return '2w'
    elif 2 <= weeks < 3:
        return '3w'
    elif 3 <= weeks < 4:
        return '4w'
    elif 4 <= weeks < 8:
        return '4w+'
    elif 8 <= weeks < 12:
        return '8w+'
    else:
        return '12w+'

In [ ]:
# Function to extract tokens and remove them from the remaining text
def extract_tokens_and_text(title):
    # Updated list of tokens to check for, excluding 'trainee'
    tokens = ['Scientist','Engineer','Thesis worker', 'Researcher', 'Data Steward', 'Consultant',
          'Analyst', 'manager', 'modeler', 'Specialist', 'Developer',
          'Officer', 'Product Owner', 'Architect', 'Ethicist','Internship']
    # List of stop words to remove from remaining text
    stop_words = {'in', 'for', 'and', 'of', 'the', 'to', 'with', 'a', 'on', 'by'}

    title_lower = title.lower()
    matched_tokens = [token for token in tokens if token.lower() in title_lower]

    # Remove matched tokens from the title to create the remaining text
    for token in matched_tokens:
        title = re.sub(re.escape(token), '', title, flags=re.IGNORECASE).strip()

    # Filter out stop words from the remaining text
    remaining_words = [word for word in re.findall(r'\b\w+\b', title) if word.lower() not in stop_words]
    remaining_text = ' '.join(remaining_words)

    return matched_tokens, remaining_text

In [ ]:
# Function to split text by commas and handle stop words
def split_and_filter(text):
    stop_words = {'in', 'for', 'and', 'the', 'of', 'to', 'with', 'a', 'on', 'by'}
    if isinstance(text, str):  # Ensure text is a string
        # Split by comma and 'and'
        parts = text.split(',')
        split_words = []
        for part in parts:
            # Split by 'and'
            sub_parts = part.split('and')
            for sub_part in sub_parts:
                # Remove stop words and leading/trailing spaces
                cleaned_part = ' '.join(word.strip() for word in sub_part.split() if word.lower() not in stop_words)
                if cleaned_part:
                    split_words.append(cleaned_part)
        return split_words
    return []  # Return an empty list if text is not a string


In [ ]:
import pandas as pd

def data_profiling(df, duplicates_path='duplicates.csv', summary_path='before_cleaning.txt'):
    """
    Perform data profiling on the given DataFrame, save duplicates, and drop them from the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.
        col1 (str): The first column to check for duplicates.
        col2 (str): The second column to check for duplicates.
        duplicates_path (str): The path to save duplicates for investigation.
        summary_path (str): The path to save the summary of metrics.

    Returns:
        pd.DataFrame: A DataFrame containing the cleaned dataset with non-null company URLs.
    """

    profile_summary = []

    # Identify duplicates based on the specified columns
    duplicates = df[df.duplicated(subset=['title', 'companyName'], keep=False)]  # Keep all duplicates
    duplicate_count = duplicates.shape[0]

    # Save duplicates to CSV for investigation
    duplicates.to_csv(duplicates_path, index=False)

    # Count null values and store detailed info
    null_counts = df.isnull().sum()
    for col, count in null_counts[null_counts > 0].items():
        profile_summary.append([f'Null count of {col}', count])  # Append each column's null count

    # Count missing values (NaN and None)
    missing_counts = df.isna().sum()
    for col, count in missing_counts[missing_counts > 0].items():
        profile_summary.append([f'Missing count of {col}', count])  # Append each column's missing count

    # Summary of dataset shape
    profile_summary.append(['Shape', df.shape])  # Append shape as a tuple
    profile_summary.append(['Duplicate Count', duplicate_count])  # Append duplicate count

    # Create a vertical summary DataFrame
    vertical_summary = pd.DataFrame(profile_summary, columns=['Metric', 'Value'])

    # Save the summary to a text file
    with open(summary_path, 'w') as f:
        for index, row in vertical_summary.iterrows():
            f.write(f"{row['Metric']}: {row['Value']}\n")

    # Drop duplicates from the DataFrame
    df_cleaned = df.drop_duplicates(subset=['title', 'companyName'], keep='first')  # Keep the first occurrence

    # Filter rows where 'companyUrl' is not null
    df_cleaned = df_cleaned[df_cleaned['companyUrl'].notnull()]

    return df_cleaned

# Example of using the function
# df = pd.read_csv('your_dataset.csv')  # Load your DataFrame
# cleaned_df = data_profiling(df, duplicates_path='duplicates.csv', summary_path='before_cleaning.txt')
# cleaned_df.to_csv('cleaned_dataset.csv', index=False)  # Save the cleaned DataFrame


In [ ]:
#Unknown value count check
import pandas as pd

def count_unknown_values_in_csv(file_path, columns):
    """
    Count the number of 'unknown' values in specified columns of a CSV file.

    Parameters:
    - file_path (str): The path to the CSV file.
    - columns (list of str): The list of column names to search for 'unknown' values.

    Returns:
    - dict: A dictionary with column names as keys and counts of 'unknown' as values.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Initialize dictionary to store counts
    unknown_counts = {}

    # Iterate through specified columns and count 'unknown' values
    for column in columns:
        if column in df.columns:
            unknown_counts[column] = df[column].str.lower().eq('unknown').sum()
        else:
            unknown_counts[column] = 'Column not found'

    return unknown_counts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
